# Homework

The idea of this homework is to create a a LLM based application(RAG, smart prompt engineering, etc) by utilizing LLMOps tools like LangChain, Llamaindex and Langfuse.


In [1]:
# Installing the required packages
!pip install langchain langchain_community boto3 langfuse llama-index-core llama-index pymupdf langchain-chroma

### Langchain

In [2]:
# Importing all the neccessary modules/libraries
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings
from langchain_chroma import Chroma
from langfuse.callback import CallbackHandler

In [3]:
#!wget https://www.researchgate.net/profile/Juan-Barrios-22/publication/374840957_Detecting_Psychological_Disorders_with_Stylometry_the_Case_of_ADHD_in_Adolescent_Autobiographical_Narratives/links/653153c95d51a8012b54b53c/Detecting-Psychological-Disorders-with-Stylometry-the-Case-of-ADHD-in-Adolescent-Autobiographical-Narratives.pdf -O stylometry.pdf

In [4]:
#!wget https://www.researchgate.net/publication/384411176_Development_and_Enlightenment_of_Artificial_Intelligence_Writing_to_Linguistics_--A_Case_Study_of_ChatGPT/fulltext/66f7b16ef599e0392fa93a9e/Development-and-Enlightenment-of-Artificial-Intelligence-Writing-to-Linguistics--A-Case-Study-of-ChatGPT.pdf -O linguistic_genAI.pdf

In [5]:
#!wget https://www.researchgate.net/profile/Adibeh-Rahmani-2/publication/370155471_The_global_prevalence_of_ADHD_in_children_and_adolescents_a_systematic_review_and_meta-analysis/links/6475eac36fb1d1682b1c4275/The-global-prevalence-of-ADHD-in-children-and-adolescents-a-systematic-review-and-meta-analysis.pdf -O adhd_prevalence.pdf

In [1]:
import os
from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [7]:
# Defining the configuration
REGION_NAME = "us-west-2"
CREDENTIALS_PROFILE_NAME = "ML"

PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
HOST = os.getenv("LANGFUSE_HOST")

EMBEDDER_MODEL_ID = "amazon.titan-embed-text-v2:0"
EMBEDDER_MODEL_KWARGS = {
    "dimensions": 512,
    "normalize": True
}

LLM_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0" # anthropic.claude-3-haiku-20240307-v1:0 or anthropic.claude-3-sonnet-20240229-v1:0 or anthropic.claude-v2:1
LLM_MODEL_KWARGS = {
    "max_tokens": 4096,
    "temperature": 0.1,
    "top_p": 1,
    "top_k": 250,
    "stop_sequences": ["\n\nHuman"]
}

CHUNK_SIZE = 2000
CHUNK_OVERLAP = 100

DATA_PATHS = [
    "stylometry.pdf",
    "linguistic_genAI.pdf",
    "adhd_prevalence.pdf"
]

VECTOR_STORE_PATH = "./vector_database/"

SEARCH_TYPE = "similarity"
RETRIEVER_KWARGS = {
    "k": 5
}

VECTOR_STORE_PATH = "./vector_database/"

INPUT_KEY = "question"
MEMORY_KEY = "history"
INPUT_VARIABLES = ["context", "history", "question"]

# Inside in the prompt template, you can play with the system's persona, the context, the history, and the question.
PROMPT_TEMPLATE = """
System: You are a helpful, respectful and honest assistant for Machine Learning.
Always answer as helpfully as possible, while being safe.
Please ensure that your responses are socially unbiased and positive in nature.
When addressing the user, always base your responses on the context provided and the previous chat history if its available.
If you are unsure about the answer, please let the user know.
If the user asks something that is not related to Machine Learning, please let the user know.
Human:
----------
<context>
{context}
</context>
----------
<history>
{history}
</history>
----------
<question>
{question}
</question>
----------
Assistant:
"""

In [8]:
# Defining the chunker
splitter = RecursiveCharacterTextSplitter(
chunk_size=CHUNK_SIZE,
chunk_overlap=CHUNK_OVERLAP
)

In [9]:
# Creating chunks from the documents
global_chunks = []
for data_path in DATA_PATHS:
    loader = PyMuPDFLoader(os.path.join(os.getcwd(), data_path))
    docs = loader.load()
    chunks = splitter.split_documents(docs)
    global_chunks.extend(chunks)

In [10]:
# Creating the embedder
embedder = BedrockEmbeddings(
    model_id=EMBEDDER_MODEL_ID,
    model_kwargs=EMBEDDER_MODEL_KWARGS,
    region_name=REGION_NAME
)

/var/folders/w0/r0k6chkn2nzd_g1c2pdl2pf40000gn/T/ipykernel_76011/3512354241.py:2: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  embedder = BedrockEmbeddings(


In [11]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedder,
    persist_directory=VECTOR_STORE_PATH
)

In [12]:
# Creating the LLM and Embedder models
llm = BedrockChat(region_name=REGION_NAME, model_id=LLM_MODEL_ID, model_kwargs=LLM_MODEL_KWARGS)

/var/folders/w0/r0k6chkn2nzd_g1c2pdl2pf40000gn/T/ipykernel_76011/3606048689.py:2: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import ChatBedrock``.
  llm = BedrockChat(region_name=REGION_NAME, model_id=LLM_MODEL_ID, model_kwargs=LLM_MODEL_KWARGS)


In [13]:
db = Chroma(persist_directory=VECTOR_STORE_PATH, embedding_function=embedder)
retriever = db.as_retriever(search_type=SEARCH_TYPE, **RETRIEVER_KWARGS)

In [14]:
# Creating the memory and the prompt template
memory = ConversationBufferWindowMemory(memory_key=MEMORY_KEY, input_key=INPUT_KEY, k=3, ai_prefix="Assistant")
prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=INPUT_VARIABLES)

/var/folders/w0/r0k6chkn2nzd_g1c2pdl2pf40000gn/T/ipykernel_76011/3752146924.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(memory_key=MEMORY_KEY, input_key=INPUT_KEY, k=3, ai_prefix="Assistant")


In [15]:
# Creating the callback handler
langfuse_callback= CallbackHandler(
        public_key=PUBLIC_KEY,
        secret_key=SECRET_KEY,
        host=HOST,
    )

In [16]:
# Creating the Chain for usage
chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever,
            verbose=True,
            return_source_documents=True,
            chain_type_kwargs={
                "prompt": prompt,
                "memory": memory
            }
)
response = chain.invoke("What did the french researchers do in order to diagnose ADHD?", config={"callbacks": [langfuse_callback]})



> Entering new RetrievalQA chain...

> Finished chain.


In [17]:
print(response["result"])

The passage does not mention any French researchers or specific methods they used to diagnose ADHD. This appears to be a systematic review and meta-analysis study that examined the prevalence of ADHD in children and adolescents based on various diagnostic criteria (DSM-V, DSM-IV, DSM-IV-TR, ICD-10) across previously published cross-sectional observational studies. The authors searched multiple databases like PubMed, Scopus, Web of Science etc. to identify and include relevant studies in their analysis, but no specific details are provided about French researchers or their diagnostic approaches.


### llama-index

In [18]:
%pip install llama-index-llms-openai
%pip install llama-index-llms-langchain
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
from llama_index.core import Document
from llama_index.core.node_parser import SimpleNodeParser
import fitz

parser = SimpleNodeParser(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

global_chunks = []
for data_path in DATA_PATHS:
    with fitz.open(data_path) as pdf_document:
        text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text("text")

        chunks = parser.split_texts([text])
        docs = [Document(text=chunk) for chunk in chunks]
        global_chunks.extend(docs)


In [20]:
from llama_index.embeddings.bedrock import BedrockEmbedding

embed_model = BedrockEmbedding(
    model_name= EMBEDDER_MODEL_ID,
    region_name=REGION_NAME,
    kwargs=EMBEDDER_MODEL_KWARGS
)

In [21]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex, StorageContext

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("homework-w8")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents=global_chunks, storage_context=storage_context, embed_model=embed_model
)

In [22]:
from llama_index.llms.bedrock import Bedrock

llm = Bedrock(
    model=LLM_MODEL_ID,
    region_name=REGION_NAME,
    kwargs=LLM_MODEL_KWARGS
)

In [23]:
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("homework-w8")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

In [24]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.prompts.base import ChatPromptTemplate

qa_prompt_str = """
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
    """

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """You are a helpful, respectful and honest assistant for Machine Learning.
            Always answer as helpfully as possible, while being safe.
            Please ensure that your responses are socially unbiased and positive in nature.
            When addressing the user, always base your responses on the context provided and the previous chat history if its available.
            If you are unsure about the answer, please let the user know.
            If the user asks something that is not related to Machine Learning, please let the user know."""
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]


text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
query_engine = index.as_query_engine(llm=llm, text_qa_template=text_qa_template, search_type=SEARCH_TYPE, **RETRIEVER_KWARGS)

In [25]:
from langfuse.llama_index import LlamaIndexInstrumentor
 
instrumentor = LlamaIndexInstrumentor()
instrumentor.start()

result = query_engine.query("What did the french researchers do in order to diagnose ADHD?")

instrumentor.flush()

In [26]:
result

Response(response='Based on the context information provided, the French researchers followed these steps to diagnose ADHD in the participants:\n\n1. For the ADHD group, adolescents with ADHD were recruited as part of a research project at the Unit of Developmental Clinical Psychology at the University of Geneva.\n\n2. Diagnostic criteria for ADHD were investigated through detailed anamnestic interviews.\n\n3. The diagnosis was confirmed by a semi-structured parents interview using the ADHD Child Evaluation (reference [22] in the text).\n\n4. All diagnostic assessments were conducted by experienced clinical psychologists specialized in ADHD.\n\nSo in summary, they used a combination of detailed interviews with the adolescents and their parents, along with a standardized ADHD evaluation instrument (ADHD Child Evaluation), administered by experienced clinical psychologists, to diagnose ADHD in the participants included in the ADHD group.', source_nodes=[NodeWithScore(node=TextNode(id_='1